# Clustering Crypto

In [101]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [102]:
# Load the crypto_data.csv dataset.
csv = 'crypto_data.csv'
df = pd.read_csv(csv, index_col='Unnamed: 0')
df.sample(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
QBT,Cubits,SHA-256,True,PoW/PoS,NaN,300000000
DNR,Denarius,NIST5,True,PoW/PoS,4.171382e+06,10000000
MUU,MilkCoin,SHA-256,True,PoW/PoS,NaN,4500000000
LC,Lutetium Coin,X11,True,PoS,6.570000e+08,657000000
RUP,Rupee,Lyra2REv2,True,PoS,2.400000e+07,24000000
RYC,RoyalCoin,Scrypt,True,PoW,NaN,140000000
TAB,MollyCoin,Scrypt,True,PoW/PoS,NaN,25000000
DYN,Dynamic,Argon2d,True,PoW,1.978422e+07,0
PIZZA,PizzaCoin,X11,True,PoW,1.377917e+06,25000000
BTCC,Bitcoin Core,SHA-256,True,PoW,NaN,21000000


In [103]:
df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [104]:
# Keep all the cryptocurrencies that are being traded.
df = df[(df.IsTrading == True)]
df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [105]:
# Remove the "IsTrading" column. 
df = df.drop(columns=['IsTrading'])
df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [106]:
# Remove rows that have at least 1 null value.
df = df.dropna()
print(df.shape)
df

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [107]:
df.TotalCoinsMined.min()

-5917977547.96773

In [108]:
# Keep the rows where coins are mined.
df = df[(df.TotalCoinsMined > 0)]
print(df.shape)
df

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [109]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df = pd.DataFrame(df.CoinName)
coin_name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [110]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df = df.drop(columns=['CoinName'])
print(df.shape)
df.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [111]:
df.TotalCoinSupply=df.TotalCoinSupply.astype(float)

In [112]:
col_strings = list(df.select_dtypes(include='object'))
col_strings

['Algorithm', 'ProofType']

In [113]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(df, columns=col_strings)
print(X.shape)
X.head()

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(df_encoded)
print(X_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [115]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3, random_state=1)
crypto_pca = pca.fit_transform(X_scaled)

In [116]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index = X.index
)
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.334216,1.059431,-0.611129
404,-0.317579,1.060258,-0.611764
1337,2.299018,1.740704,-0.746129
BTC,-0.141841,-1.332626,0.202796
ETH,-0.149732,-1.990729,0.406060
LTC,-0.164912,-1.058848,-0.008733
DASH,-0.398149,1.182077,-0.499508
XMR,-0.155836,-2.253483,0.502534
ETC,-0.148178,-1.990762,0.406026
ZEC,-0.156200,-2.110309,0.399649


In [117]:
pca.explained_variance_ratio_

array([0.02793106, 0.02133172, 0.02049874])

In [118]:
# Fetch the explained variance
ev = sum(pca.explained_variance_ratio_)
print(f'The explained variance with 3 principal components is: {ev:.2f}')

The explained variance with 3 principal components is: 0.07


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [119]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

C:\Users\jonat\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [120]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [121]:
coin_name_df = coin_name_df.set_index(X.index)
coin_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [122]:
# Initialize the K-means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Add the predicted class columns
coin_name_df["class"] = model.labels_
coin_name_df.sample(10)

,CoinName,class
FRED,FREDEnergy,3
CREDIT,Credit,0
UMO,Universal Molecule,3
SXC,SexCoin,3
GPU,GPU Coin,0
CRYP,CrypticCoin,3
RBY,RubyCoin,0
HXX,HexxCoin,3
BTDX,Bitcloud 2.0,0
BOLI,BolivarCoin,3


In [124]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pcs_df, coin_name_df], axis=1)

print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.334216,1.059431,-0.611129,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.317579,1.060258,-0.611764,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.299018,1.740704,-0.746129,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.141841,-1.332626,0.202796,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.149732,-1.990729,0.406060,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.164912,-1.058848,-0.008733,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.398149,1.182077,-0.499508,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.155836,-2.253483,0.502534,Monero,3
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.148178,-1.990762,0.406026,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.156200,-2.110309,0.399649,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [142]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data=['Algorithm'],
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [143]:
clustered_df.columns

Index(['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply', 'PC 1',
       'PC 2', 'PC 3', 'CoinName', 'class'],
      dtype='object')

In [144]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'],
                          sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [146]:
# Print the total number of tradable cryptocurrencies.
tradable = clustered_df.CoinName.count()
print(f'The total number of tradable cryptocurrencies is: {tradable}')

The total number of tradable cryptocurrencies is: 532


In [147]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
coins_df = clustered_df[['TotalCoinsMined', 'TotalCoinSupply']]
coins_df

,TotalCoinsMined,TotalCoinSupply
42,4.199995e+01,4.200000e+01
404,1.055185e+09,5.320000e+08
1337,2.927942e+10,3.141593e+11
BTC,1.792718e+07,2.100000e+07
ETH,1.076842e+08,0.000000e+00
...,...,...
ZEPH,2.000000e+09,2.000000e+09
GAP,1.493105e+07,2.500000e+08
BDX,9.802226e+08,1.400223e+09
ZEN,7.296538e+06,2.100000e+07


In [150]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
coins_df_scaled = pd.DataFrame(MinMaxScaler().fit_transform(coins_df))
coins_df_scaled = coins_df_scaled.set_index(clustered_df.index)

In [152]:
coins_df_scaled.describe()

,0,1
count,532.000000,532.000000
mean,0.005394,0.010345
std,0.046107,0.067737
min,0.000000,0.000000
25%,0.000008,0.000021
50%,0.000025,0.000078
75%,0.000189,0.000500
max,1.000000,1.000000


In [163]:
plot_df = coins_df_scaled.copy()

CN = clustered_df.CoinName
CN
class_num = clustered_df['class']

plot_df = pd.concat([plot_df,CN,class_num], axis=1)

In [164]:
plot_df.head(10)

,0,1,CoinName,class
42,0.000000,4.200000e-11,42 Coin,0
404,0.001066,5.320000e-04,404Coin,0
1337,0.029576,3.141593e-01,EliteCoin,0
BTC,0.000018,2.100000e-05,Bitcoin,3
ETH,0.000109,0.000000e+00,Ethereum,3
LTC,0.000064,8.400000e-05,Litecoin,3
DASH,0.000009,2.200000e-05,Dash,0
XMR,0.000017,0.000000e+00,Monero,3
ETC,0.000115,2.100000e-04,Ethereum Classic,3
ZEC,0.000007,2.100000e-05,ZCash,3


In [165]:
plot_df.columns = ['Total Coins Mined', 'Total Coin Supply','Coin Name', 'Class']
plot_df.head()

,Total Coins Mined,Total Coin Supply,Coin Name,Class
42,0.000000,4.200000e-11,42 Coin,0
404,0.001066,5.320000e-04,404Coin,0
1337,0.029576,3.141593e-01,EliteCoin,0
BTC,0.000018,2.100000e-05,Bitcoin,3
ETH,0.000109,0.000000e+00,Ethereum,3


In [173]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x='Total Coins Mined',
    y='Total Coin Supply',
    by='Class',
    hover_cols = ['Coin Name'],
    size = 100,
    alpha = 0.75,
    
    
)

:NdOverlay   [Class]
   :Scatter   [Total Coins Mined]   (Total Coin Supply,Coin Name)